In [5]:
#!/usr/bin/env python3
"""
trade_analysis.py – Analyse & Auswerten deiner Trading‑Bot‑Performance
====================================================================
Dieses Skript lässt sich sowohl als **Standalone‑Tool** (CLI) als auch als
**Notebook‑Modul** nutzen – ohne dass du zwangsweise alle Argumente
angeben musst.

Änderungen (v0.2)
-----------------
* **--quote_balance** ist jetzt **optional**. Wird es weggelassen, nimmt das
  Skript `0.0` USDT als Start‑Equity und weist per Warnung darauf hin.
* Besseres Fehlermanagement beim Aufruf ohne Parameter – damit blendet sich
  das Script problemlos in Positron/Jupyter ein.

Quick Start
-----------
```bash
# CSV auswerten
python trade_analysis.py --file trades.csv --quote_balance 944.8236

# Wenn du --quote_balance weglässt, startet die Equity‑Kurve bei 0 USDT
python trade_analysis.py --file trades.csv

# Live‑Analyse (API‑Keys müssen in ENV‑Variablen stehen)
python trade_analysis.py --live --quote_balance 944.8236
```

Notebook‑Nutzung
----------------
```python
from trade_analysis import run_analysis
trades, stats = run_analysis(file="trades.csv", quote_balance=944.8236)
print(stats)
```
"""
from __future__ import annotations

import argparse
import os
from pathlib import Path
from typing import Optional, Tuple

import pandas as pd
import numpy as np

# Versuche Bitget‑SDK zu laden (optional)
try:
    from bitget.spot import Spot  # type: ignore
except ImportError:
    Spot = None  # noqa: N816


# ---------------------------------------------------------------------------
# Datenquellen
# ---------------------------------------------------------------------------

def read_trades_from_csv(path: Path) -> pd.DataFrame:
    """Lade Trade‑CSV (Pflichtspalten siehe README) und liefere DataFrame."""
    df = pd.read_csv(path, parse_dates=["timestamp"])
    required = {"timestamp", "symbol", "side", "quantity", "price", "fee", "realized_pnl"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"CSV fehlt Spalten: {', '.join(sorted(missing))}")
    return df


def read_trades_from_bitget() -> pd.DataFrame:  # noqa: D401
    """Ziehe Fills von Bitget, wenn SDK + ENV Keys vorhanden sind."""
    if Spot is None:
        raise RuntimeError("bitget-python-sdk nicht installiert (pip install bitget)")

    api_key = os.getenv("BITGET_API_KEY")
    api_secret = os.getenv("BITGET_API_SECRET")
    api_pass = os.getenv("BITGET_PASSPHRASE")
    if not all([api_key, api_secret, api_pass]):
        raise EnvironmentError("API‑Variablen BITGET_API_KEY/_SECRET/_PASSPHRASE müssen gesetzt sein")

    client = Spot(api_key, api_secret, api_pass, use_server_time=True)
    raw = client.fills(symbol="ADAUSDT")
    df = pd.DataFrame(raw["data"])

    mapping = {
        "cTime": "timestamp",
        "side": "side",
        "size": "quantity",
        "priceAvg": "price",
        "fee": "fee",
        "profit": "realized_pnl",
    }
    df.rename(columns=mapping, inplace=True)

    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    float_cols = ["quantity", "price", "fee", "realized_pnl"]
    df[float_cols] = df[float_cols].astype(float)
    return df


# ---------------------------------------------------------------------------
# Kennzahlen berechnen
# ---------------------------------------------------------------------------

def compute_statistics(df: pd.DataFrame, quote_balance: float) -> dict:
    gross = df["realized_pnl"].sum()
    fees = df["fee"].sum()
    net = gross - fees
    wins = df[df["realized_pnl"] > 0]
    win_rate = (len(wins) / len(df) * 100) if len(df) else 0

    equity_curve = (df["realized_pnl"] - df["fee"]).cumsum() + quote_balance
    drawdown = (equity_curve.cummax() - equity_curve) / equity_curve.cummax()
    max_dd = drawdown.max() * 100 if not drawdown.empty else 0

    daily_pnl = df.groupby(df["timestamp"].dt.date)["realized_pnl"].sum()
    sharpe = (np.sqrt(365) * daily_pnl.mean() / daily_pnl.std()) if daily_pnl.std() else np.nan

    return {
        "Gross PnL": gross,
        "Fees": fees,
        "Net PnL": net,
        "Win rate %": win_rate,
        "Max DD %": max_dd,
        "Sharpe": sharpe,
    }


# ---------------------------------------------------------------------------
# Public API
# ---------------------------------------------------------------------------

def run_analysis(*, file: Optional[str] = None, live: bool = False, quote_balance: float = 0.0) -> Tuple[pd.DataFrame, dict]:
    """Liefere Trades‑DataFrame und Statistik‑Dict zurück."""
    source_df: pd.DataFrame
    if live:
        source_df = read_trades_from_bitget()
    elif file:
        source_df = read_trades_from_csv(Path(file))
    else:
        raise ValueError("Entweder --file angeben oder --live Flag setzen.")

    stats = compute_statistics(source_df, quote_balance)
    return source_df, stats


# ---------------------------------------------------------------------------
# CLI‑Eintrittspunkt
# ---------------------------------------------------------------------------

def main() -> None:  # noqa: D401
    parser = argparse.ArgumentParser(description="Analyse Trading‑Performance")
    parser.add_argument("--file", type=str, help="CSV mit Trade‑Historie")
    parser.add_argument("--quote_balance", type=float, default=0.0, help="Start‑Quote‑Balance (USDT)")
    parser.add_argument("--live", action="store_true", help="Daten via Bitget‑API beziehen")

    # parse_known_args => ignoriert Notebook‑Sonderflags (--logfile etc.)
    args, _unknown = parser.parse_known_args()

    if args.quote_balance == 0.0:
        print("[WARN] --quote_balance fehlt: Equity‑Kurve startet bei 0 USDT.\n")

    trades, stats = run_analysis(file=args.file, live=args.live, quote_balance=args.quote_balance)

    print("\n=== Trade‑Statistik ===")
    for k, v in stats.items():
        if isinstance(v, float):
            print(f"{k:<12}: {v:,.4f}")
        else:
            print(f"{k:<12}: {v}")

    print("\nErste 5 Trades:")
    print(trades.head().to_string(index=False))


if __name__ == "__main__":
    main()


[WARN] --quote_balance fehlt: Equity‑Kurve startet bei 0 USDT.



ValueError: Entweder --file angeben oder --live Flag setzen.